In [7]:
import pandas as pd
import glob
import logging
import os
import numpy as np

In [8]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger(__name__)

In [9]:
mode = "average"

In [10]:
df = pd.read_csv("big_table_by_{}.tsv".format(mode), sep="\t").dropna()

In [11]:
COLUMNS = [c for c in df.columns if '(' in c]
len(COLUMNS)

1621

In [12]:
N = 1000
BOOTSTRAP_TEMPLATE = "{}/bootstrap_{}_{}_by_genre.tsv"
OUT_TEMPLATE = "{}/significant_axes_{}_{}_by_genre.tsv"
OUT_TEMPLATE2 = "{}/effect_size_significant_axes_{}_{}_by_genre.tsv"

for genre, a_count in df['genre'].value_counts().iteritems():
    if a_count < 1000:
        continue
        
    results = []        
    logger.info("{}...".format(genre))
    
    df_actual = df.query('genre==@genre')
    df_bootstrap = pd.read_csv(BOOTSTRAP_TEMPLATE.format(mode, genre, mode), sep="\t").dropna()

    for axis in COLUMNS:
        actual = np.mean(df_actual[axis], axis=0)
        significance = sum(abs(df_bootstrap[axis]) > abs(actual))/float(N)
        results.append([axis, actual-np.mean(df_bootstrap[axis], axis=0), significance])

    pd.DataFrame(sorted(results, key=lambda x:x[2]), 
                 columns = ["axis", "diff_a_b", "p"]
                ).to_csv(OUT_TEMPLATE.format(mode, genre, mode), sep="\t", index=False)

    pd.DataFrame(sorted(results, key=lambda x:abs(x[1]), reverse=True), 
         columns = ["axis", "diff_a_b", "p"]
        ).query('p <= 0.05').to_csv(OUT_TEMPLATE2.format(mode, genre, mode), sep="\t", index=False)

2020-04-18 16:37:19 __main__     INFO     Drama...
2020-04-18 16:37:20 __main__     INFO     Comedy...
